In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

import os
from glob import glob
import pandas as pd
import numpy as np

In [ ]:
def CNNImplementation(df):

    # Set window size
    window_size = 10
    num_features = 20

    X = df.drop(columns=['stress']).values
    y = df['stress'].values

    # Iterate over data with step 1 and extract windows
    X = [X[i:i+window_size] for i in range(0, len(X) - window_size + 1, window_size)]
    y = [y[i:i+window_size] for i in range(0, len(y) - window_size + 1, window_size)]

    # Convert windows to numpy array
    X = np.array(X)
    y = np.array(y)


    out_list = []

    # Iterate over input array
    for subarr in y:
        if np.all(subarr == 1):
            out_list.append([1])
        elif np.all(subarr == 0):
            out_list.append([0])


    out_arr = []
    for x in out_list:
        out_arr.extend([x for i in range(int(X.shape[2]/num_features))])

    print("X.shape: {} and y.shape: {}".format(X.shape, y.shape))

    # Convert output list to numpy array
    out_arr = np.array(out_arr)
    y = out_arr
    X = X.reshape(-1, window_size, num_features, 1)

    print("X.shape: {} and y.shape: {}".format(X.shape, y.shape))

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)


    # Define the CNN model
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(window_size,num_features,1)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='softmax')

    ])
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data = (X_test, y_test))

    loss, accuracy = model.evaluate(X_test, y_test)

    return accuracy


In [ ]:
def AE_LSTM_implementation(df):
    X = df.drop(columns=['stress']).values
    y = df['stress'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 42)

    X_train = tf.convert_to_tensor(X_train)
    X_train = tf.expand_dims(X_train, axis=1)
    X_test = tf.convert_to_tensor(X_test)
    X_test = tf.expand_dims(X_test, axis=1)

    # Define the input shape
    input_shape = (X_train.shape[1], X_train.shape[2])

    # Define the size of the bottleneck layer
    latent_dim = 64

    # Define the encoder
    inputs = Input(shape=input_shape)
    x = LSTM(latent_dim, activation='relu')(inputs)
    n = K.int_shape(x)[1]
    encoded = RepeatVector(n)(x)

    # Define the decoder
    decoded = LSTM(input_shape[1], activation='relu', return_sequences=True)(encoded)

    # Combine the encoder and decoder into an autoencoder model
    autoencoder = Model(inputs, decoded)

    # Compile the autoencoder model
    autoencoder.compile(optimizer='adam', loss='mse')

    # Train the autoencoder model
    autoencoder.fit(X_train, X_train, epochs=1, batch_size=32, validation_split=0.1)

    # Extract the bottleneck layer
    encoder = Model(inputs, encoded)
    encoded_train_data = encoder.predict(X_train)

    # Define the LSTM classifier
    lstm_input = Input(shape=(encoded_train_data.shape[1], encoded_train_data.shape[2]))

    x = LSTM(64, return_sequences=True)(lstm_input)
    outputs = LSTM(1, activation='sigmoid', return_sequences=True)(x)

    classifier = Model(lstm_input, outputs)

    # Compile the classifier model
    classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the classifier model using the encoded training data
    classifier.fit(encoded_train_data, y_train, epochs=10, batch_size=32, validation_split=0.1)




In [ ]:
def mergeCSV(src, path):

  # Merging the files
  joined_files = os.path.join(src, path)

  # A list of all joined files is returned
  joined_list = glob(joined_files)

  # Finally, the files are joined
  df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

  # Save in specified location
  return df


In [ ]:
def apply_corr_filter(df, threshold):

    # Calculate correlation matrix
    corr_matrix = df.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(pd.np.triu(pd.np.ones(corr_matrix.shape), k=1).astype(bool))

    # Find features with correlation above threshold
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

    # Drop highly correlated features
    filtered_df = df.drop(to_drop, axis=1)

    return filtered_df


In [ ]:
location = r"/content/gdrive/My Drive/SAM40/Preprocessing_own/Preprocessed_filtered_data_features/Time_Frequency_domain"

stressdata = mergeCSV(
    location + r"/Arithmetic*",
    r"Arithmetic*.csv"
)
if 'Unnamed: 0.1' in stressdata.columns:
    stressdata.drop(['Unnamed: 0.1'], axis=1, inplace=True)
if 'Unnamed: 0' in stressdata.columns:
    stressdata.drop(['Unnamed: 0'], axis=1, inplace=True)

stressdata['stress'] = 1

relaxdata = mergeCSV(
    location + r"/Relax*",
    r"Relax*.csv"
)

if 'Unnamed: 0.1' in relaxdata.columns:
    relaxdata.drop(['Unnamed: 0.1'], axis=1, inplace=True)
if 'Unnamed: 0' in relaxdata.columns:
    relaxdata.drop(['Unnamed: 0'], axis=1, inplace=True)

relaxdata['stress'] = 0

data = pd.concat([stressdata, relaxdata])


In [ ]:
data = apply_corr_filter(data, 0.7)

<ipython-input-6-0ef0483afea9>:7: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  upper = corr_matrix.where(pd.np.triu(pd.np.ones(corr_matrix.shape), k=1).astype(bool))


In [ ]:
data.shape

(6000, 21)

In [ ]:
print(location, ": ")

print(CNNImplementation(data))

/content/gdrive/My Drive/SAM40/Preprocessing_own/Preprocessed_filtered_data_features/Time_Frequency_domain : 
X.shape: (600, 10, 20) and y.shape: (600, 10)
X.shape: (600, 10, 20, 1) and y.shape: (600, 1)
Epoch 1/10
15/15 [==============================] - 2s 35ms/step - loss: 1.0733 - accuracy: 0.4958 - val_loss: 0.8593 - val_accuracy: 0.5167
Epoch 2/10
15/15 [==============================] - 0s 18ms/step - loss: 0.6973 - accuracy: 0.4958 - val_loss: 0.7168 - val_accuracy: 0.5167
Epoch 3/10
15/15 [==============================] - 0s 15ms/step - loss: 0.7141 - accuracy: 0.4958 - val_loss: 0.7131 - val_accuracy: 0.5167
Epoch 4/10
15/15 [==============================] - 0s 17ms/step - loss: 0.6643 - accuracy: 0.4958 - val_loss: 0.7574 - val_accuracy: 0.5167
Epoch 5/10
15/15 [==============================] - 0s 17ms/step - loss: 0.6380 - accuracy: 0.4958 - val_loss: 0.7406 - val_accuracy: 0.5167
Epoch 6/10
15/15 [==============================] - 0s 16ms/step - loss: 0.6598 - accuracy:

In [ ]:
AE_LSTM_implementation(data)

141/141 [==============================] - 0s 2ms/step
Epoch 1/10
127/127 [==============================] - 14s 81ms/step - loss: 0.6982 - accuracy: 0.5069 - val_loss: 0.6944 - val_accuracy: 0.4935
Epoch 2/10
127/127 [==============================] - 8s 61ms/step - loss: 0.6940 - accuracy: 0.4996 - val_loss: 0.6931 - val_accuracy: 0.5156
Epoch 3/10
127/127 [==============================] - 9s 72ms/step - loss: 0.6937 - accuracy: 0.4975 - val_loss: 0.6932 - val_accuracy: 0.5070
Epoch 4/10
127/127 [==============================] - 9s 73ms/step - loss: 0.6934 - accuracy: 0.4932 - val_loss: 0.6931 - val_accuracy: 0.5126
Epoch 5/10
127/127 [==============================] - 8s 60ms/step - loss: 0.6933 - accuracy: 0.5006 - val_loss: 0.6932 - val_accuracy: 0.5099
Epoch 6/10
127/127 [==============================] - 10s 75ms/step - loss: 0.6933 - accuracy: 0.4995 - val_loss: 0.6932 - val_accuracy: 0.5108
Epoch 7/10
127/127 [==============================] - 9s 70ms/step - loss: 0.6933 - a

In [ ]:
    #Implementation of state of art deep learning algorithm

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def AE_LSTM_impli(df):
    X = df.drop(columns=['stress']).values
    y = df['stress'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

    # Normalize the input data
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    X_train = tf.expand_dims(X_train, axis=2)
    X_test = tf.expand_dims(X_test, axis=2)

    # Define the input shape
    input_shape = (X_train.shape[1], X_train.shape[2])

    # Define the size of the bottleneck layer
    latent_dim = 64

    # Define the encoder
    inputs = Input(shape=input_shape)
    x = LSTM(latent_dim, activation='relu')(inputs)
    encoded = RepeatVector(input_shape[0])(x)

    # Define the decoder
    decoded = LSTM(input_shape[1], activation='relu', return_sequences=True)(encoded)

    # Combine the encoder and decoder into an autoencoder model
    autoencoder = Model(inputs, decoded)

    # Compile the autoencoder model
    autoencoder.compile(optimizer='adam', loss='mse')

    # Train the autoencoder model
    autoencoder.fit(X_train, X_train, epochs=10, batch_size=32, validation_split=0.1)

    # Extract the bottleneck layer
    encoder = Model(inputs, encoded)
    encoded_train_data = encoder.predict(X_train)

    # Define the LSTM classifier
    lstm_input = Input(shape=(encoded_train_data.shape[1], encoded_train_data.shape[2]))

    x = LSTM(64, return_sequences=True)(lstm_input)
    outputs = LSTM(1, activation='sigmoid')(x)

    classifier = Model(lstm_input, outputs)

    # Compile the classifier model
    classifier.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

    # Train the classifier model using the encoded training data
    classifier.fit(encoded_train_data, y_train, epochs=10, batch_size=32, validation_split=0.1)

    # Evaluate the classifier model on the test data
    encoded_test_data = encoder.predict(X_test)
    loss, accuracy = classifier.evaluate(encoded_test_data, y_test)

    return accuracy

In [ ]:
AE_LSTM_impli(data)

Epoch 1/10
127/127 [==============================] - 9s 38ms/step - loss: 0.0676 - val_loss: 0.0510
Epoch 2/10
127/127 [==============================] - 2s 20ms/step - loss: 0.0502 - val_loss: 0.0503
Epoch 3/10
127/127 [==============================] - 3s 20ms/step - loss: 0.0499 - val_loss: 0.0499
Epoch 4/10
127/127 [==============================] - 4s 30ms/step - loss: 0.0498 - val_loss: 0.0498
Epoch 5/10
127/127 [==============================] - 3s 23ms/step - loss: 0.0495 - val_loss: 0.0496
Epoch 6/10
127/127 [==============================] - 3s 21ms/step - loss: 0.0493 - val_loss: 0.0497
Epoch 7/10
127/127 [==============================] - 3s 20ms/step - loss: 0.0492 - val_loss: 0.0493
Epoch 8/10
127/127 [==============================] - 3s 21ms/step - loss: 0.0490 - val_loss: 0.0491
Epoch 9/10
127/127 [==============================] - 4s 31ms/step - loss: 0.0489 - val_loss: 0.0491
Epoch 10/10
141/141 [==============================] - 1s 5ms/step
Epoch 1/10
127/127 [====

0.4873333275318146

In [ ]:
def CNNImpli(df):

    # Set window size
    window_size = 10
    num_features = 20

    X = df.drop(columns=['stress']).values
    y = df['stress'].values

    # Iterate over data with step 1 and extract windows
    X = [X[i:i+window_size] for i in range(0, len(X) - window_size + 1, window_size)]
    y = [y[i:i+window_size] for i in range(0, len(y) - window_size + 1, window_size)]

    # Convert windows to numpy array
    X = np.array(X)
    y = np.array(y)


    out_list = []

    # Iterate over input array
    for subarr in y:
        if np.all(subarr == 1):
            out_list.append([1])
        elif np.all(subarr == 0):
            out_list.append([0])


    out_arr = []
    for x in out_list:
        out_arr.extend([x for i in range(int(X.shape[2]/num_features))])

    print("X.shape: {} and y.shape: {}".format(X.shape, y.shape))

    # Convert output list to numpy array
    out_arr = np.array(out_arr)
    y = out_arr
    X = X.reshape(-1, window_size, num_features, 1)

    print("X.shape: {} and y.shape: {}".format(X.shape, y.shape))

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)


    # Define the CNN model
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(window_size,num_features,1)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')

    ])
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data = (X_test, y_test))

    loss, accuracy = model.evaluate(X_test, y_test)

    return accuracy

In [ ]:
CNNImpli(data)

X.shape: (600, 10, 20) and y.shape: (600, 10)
X.shape: (600, 10, 20, 1) and y.shape: (600, 1)
Epoch 1/10
15/15 [==============================] - 2s 92ms/step - loss: 1.3138 - accuracy: 0.5333 - val_loss: 0.7026 - val_accuracy: 0.5500
Epoch 2/10
15/15 [==============================] - 1s 50ms/step - loss: 0.7256 - accuracy: 0.5708 - val_loss: 0.7424 - val_accuracy: 0.5083
Epoch 3/10
15/15 [==============================] - 1s 43ms/step - loss: 0.6970 - accuracy: 0.5604 - val_loss: 0.8587 - val_accuracy: 0.5250
Epoch 4/10
15/15 [==============================] - 1s 47ms/step - loss: 0.6754 - accuracy: 0.5750 - val_loss: 0.7278 - val_accuracy: 0.5750
Epoch 5/10
15/15 [==============================] - 1s 48ms/step - loss: 0.6429 - accuracy: 0.6000 - val_loss: 0.7015 - val_accuracy: 0.5500
Epoch 6/10
15/15 [==============================] - 1s 50ms/step - loss: 0.6332 - accuracy: 0.6208 - val_loss: 0.7640 - val_accuracy: 0.5833
Epoch 7/10
15/15 [==============================] - 1s 39ms/

0.5666666626930237

In [ ]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 10.2 MB/s eta 0:00:00


In [ ]:
from tensorflow.keras.layers.experimental import GraphConv

ImportError: ignored

In [ ]:
def AE_CNN_GNN_implementation(df):
  # Set window size
            window_size = 10
            num_features = 20

            X = df.drop(columns=['stress']).values
            y = df['stress'].values

            # Iterate over data with step 1 and extract windows
            X = [X[i:i+window_size] for i in range(0, len(X) - window_size + 1, window_size)]
            y = [y[i:i+window_size] for i in range(0, len(y) - window_size + 1, window_size)]

            # Convert windows to numpy array
            X = np.array(X)
            y = np.array(y)


            out_list = []

            # Iterate over input array
            for subarr in y:
                if np.all(subarr == 1):
                    out_list.append([1])
                elif np.all(subarr == 0):
                    out_list.append([0])


            out_arr = []
            for x in out_list:
                out_arr.extend([x for i in range(int(X.shape[2]/num_features))])

            print("X.shape: {} and y.shape: {}".format(X.shape, y.shape))

            # Convert output list to numpy array
            out_arr = np.array(out_arr)
            y = out_arr
            X = X.reshape(-1, window_size, num_features, 1)

            print("X.shape: {} and y.shape: {}".format(X.shape, y.shape))

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)


            # Define the CNN model
            model = tf.keras.Sequential([
                tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(window_size,num_features,1)),
                tf.keras.layers.MaxPooling2D((2, 2)),
                tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
                tf.keras.layers.MaxPooling2D((2, 2)),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(64, activation='relu'),
                tf.keras.layers.Dense(1, activation='sigmoid')

            ])
            # Add a graph neural network layer
            model.add(tf.keras.layers.GraphConv(64, (3, 3)))

            # Compile the model
            model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

            # Train the model
            model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data = (X_test, y_test))

            loss, accuracy = model.evaluate(X_test, y_test)

            return accuracy


In [ ]:
AE_CNN_GNN_implementation(data)

X.shape: (600, 10, 20) and y.shape: (600, 10)
X.shape: (600, 10, 20, 1) and y.shape: (600, 1)


AttributeError: ignored

In [ ]:
def CNN_RNN_Impli(df):

    # Set window size
    window_size = 10
    num_features = 20

    X = df.drop(columns=['stress']).values
    y = df['stress'].values

    # Iterate over data with step 1 and extract windows
    X = [X[i:i+window_size] for i in range(0, len(X) - window_size + 1, window_size)]
    y = [y[i:i+window_size] for i in range(0, len(y) - window_size + 1, window_size)]

    # Convert windows to numpy array
    X = np.array(X)
    y = np.array(y)

    X = X[:, np.newaxis]
    # out_list = []

    # # Iterate over input array
    # for subarr in y:
    #     if np.all(subarr == 1):
    #         out_list.append([1])
    #     elif np.all(subarr == 0):
    #         out_list.append([0])


    # out_arr = []
    # for x in out_list:
    #     out_arr.extend([x for i in range(int(X.shape[2]/num_features))])

    # print("X.shape: {} and y.shape: {}".format(X.shape, y.shape))

    # # Convert output list to numpy array
    # out_arr = np.array(out_arr)
    # y = out_arr
    # X = X.reshape(-1, window_size, num_features, 1)

    # print("X.shape: {} and y.shape: {}".format(X.shape, y.shape))

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)


    # Define the CNN model
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(window_size,num_features,1)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    # Define the LSTM model
    lstm_model = tf.keras.Sequential([
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    # Combine the CNN and LSTM models
    model = tf.keras.Sequential([
        model,
        lstm_model
    ])

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data = (X_test, y_test))

    loss, accuracy = model.evaluate(X_test, y_test)

    return accuracy

In [ ]:
CNN_RNN_Impli(data)

ValueError: ignored